Classify the Size_Categorie using SVM

Attributes:

month month of the year: 'jan' to 'dec'

day :: day of the week: 'mon' to 'sun'

FFMC ::FFMC index from the FWI system: 18.7 to 96.20

DMC :: DMC index from the FWI system: 1.1 to 291.3

DC ::DC index from the FWI system: 7.9 to 860.6

ISI ::ISI index from the FWI system: 0.0 to 56.10

temp:: temperature in Celsius degrees: 2.2 to 33.30

RH ::relative humidity in %: 15.0 to 100

wind ::wind speed in km/h: 0.40 to 9.40

rain ::outside rain in mm/m2 : 0.0 to 6.4

Size_Categorie the burned area of the forest ( Small , Large)

In [5]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.preprocessing import StandardScaler , LabelEncoder
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [6]:
#Loding data
salary_train = pd.read_csv('SalaryData_Train(1).csv')
salary_train 

FileNotFoundError: [Errno 2] No such file or directory: 'SalaryData_Train(1).csv'

In [ ]:
salary_test = pd.read_csv('D:/ExcelR/Assignment/Support Vector Machines/SalaryData_Test(1).csv')
salary_test 

Conventionally, We perform EDA, visualisation and then split data into train and test. However, here we have train and test sets in first place only.. so we will merge them, perform EDA and visualisation.. and then again split data in order to apply ML algorithms.

In [ ]:
#concatenate Train and Test datasets
salary = salary_train.append(salary_test)
salary.reset_index(inplace=True)
salary = salary.drop(['index'], axis=1) 
salary.tail()

Descriptive statistics

In [ ]:
salary.info()

In [ ]:
salary.describe()

In [ ]:
# Missing values
salary.isna().sum().sum()

In [ ]:
#Types of variables

numeric_features = [feature for feature in salary.columns if salary[feature].dtypes != 'O']
print('numeric features:', numeric_features, '\n')

In [ ]:
#under numeric features there are 2 types of features ie. discrete and categorical

discrete_features = [feature for feature in numeric_features if len(salary[feature].unique())<25]
print('discrete features:', discrete_features, '\n')

continuous_features = [feature for feature in numeric_features if feature not in discrete_features]
print('continuous features:', continuous_features, '\n')

In [ ]:
#categorical_features

categorical_features = [feature for feature in salary.columns if salary[feature].dtypes == 'O']
print('categorical_features:', categorical_features) 

In [ ]:
# frequency counts of values in categorical variables

for feature in categorical_features:
    print(salary[feature].value_counts())

In [ ]:
#cardinality in categorical variables

for feature in categorical_features:
    print(feature, 'has', len(salary[feature].unique()), 'labels')

Exploratory Data Analysis

In [ ]:
# for continuous features -- Boxplot
fig, axes=plt.subplots(4,1,figsize=(14,8))

sns.boxplot(salary.age,  ax=axes[0])
sns.boxplot(salary.capitalgain,data=salary ,  ax=axes[1])
sns.boxplot(salary.capitalloss,  ax=axes[2])
sns.boxplot(salary.hoursperweek,  ax=axes[3])

There are lot of outliers present in the dataframe but we can't drop them because they are present in a very large quantity and can be important for model building

In [ ]:
# for categorical_features -- barplot
fig, axes=plt.subplots(len(categorical_features),1,figsize=(14,50))

for i in range (0, len(categorical_features)):
    sns.countplot(salary[categorical_features[i]],ax=axes[i], order=salary[categorical_features[i]].value_counts().index)

Univariate analysis

In [ ]:
salary_more_than_50=pd.DataFrame()
salary_less_than_50=pd.DataFrame()

salary_more_than_50 = salary.loc[salary['Salary']==' >50K']

salary_less_than_50 = salary.loc[salary['Salary']==' <=50K']

In [ ]:
salary.shape

In [ ]:
(salary_less_than_50.shape[0] + salary_more_than_50.shape[0]) ==salary.shape[0] 

Age distribution wrt salary

In [ ]:

plt.figure(figsize=(15,14))

plt.subplot(3,1,1)
plt.hist(salary_more_than_50.age, bins=[10, 20, 30, 40, 50, 60, 70, 80, 90])
plt.title('Age of those with salary_more_than_50k')

plt.subplot(3,1,2)
plt.hist(salary_less_than_50.age, color = "orange", bins=[10, 20, 30, 40, 50, 60, 70, 80, 90])
plt.title('Age of those with salary_less_than_50k')


age_describe = pd.DataFrame()

age_describe['Age of those with salary_more_than_50k'] = salary_more_than_50['age'].describe()
age_describe['Age of those with salary_less_than_50k'] = salary_less_than_50['age'].describe()

age_describe  = age_describe.drop(['count', '25%', '50%', '75%'])
plt.subplot(3,1,2)
age_describe.plot.bar()

plt.show()

Insight:

Age of those with salary more than 50K: Most of the people are in the age group of 30 - 50 and no one is below 20yrs

Age of those with salary less than 50K: As age increases, no of people are decreasing.

Hours/week distribution wrt salary

In [ ]:
plt.figure(figsize=(15,14))

plt.subplot(3,1,1)
plt.hist(salary_more_than_50.hoursperweek)
plt.title('Hours/week of those with salary_more_than_50k')

plt.subplot(3,1,2)
plt.hist(salary_less_than_50.hoursperweek, color = "orange")
plt.title('Hours/week of those with salary_less_than_50k')


Hoursweek_describe = pd.DataFrame()

Hoursweek_describe['Hours/week of those with salary_more_than_50k'] = salary_more_than_50['hoursperweek'].describe()
Hoursweek_describe['Hours/week of those with salary_less_than_50k'] = salary_less_than_50['hoursperweek'].describe()

Hoursweek_describe = Hoursweek_describe.drop(['count', '25%', '50%', '75%'])
plt.subplot(3,1,2)
Hoursweek_describe.plot.bar()

plt.show() 

nsight:

For both groups, majority is working for 30 to 40 hrs per week, and people working for 80 to 90 hrs is present in both groups

Capital Gains distribution wrt salary

In [ ]:
plt.figure(figsize=(15,14))

plt.subplot(3,1,1)
plt.hist(salary_more_than_50.capitalgain)
plt.title('capitalgain of those with salary_more_than_50k')

plt.subplot(3,1,2)
plt.hist(salary_less_than_50.capitalgain, color = "orange")
plt.title('capitalgain of those with salary_less_than_50k')


capitalgain_describe = pd.DataFrame()

capitalgain_describe['capitalgain of those with salary_more_than_50k'] = salary_more_than_50['capitalgain'].describe()
capitalgain_describe['capitalgain of those with salary_less_than_50k'] = salary_less_than_50['capitalgain'].describe()

capitalgain_describe = capitalgain_describe.drop(['count', '25%', '50%', '75%'])
plt.subplot(3,1,2)
capitalgain_describe.plot.bar()

plt.show() 

Insight:

Salary more than 50k: Capital gain for majority is between 0 to 1000

Salary less than 50k: capital gain for majority is less than 500

Capital Loss distribution wrt salary



In [ ]:
plt.figure(figsize=(15,14))

plt.subplot(3,1,1)
plt.hist(salary_more_than_50.capitalloss)
plt.title('capitalloss of those with salary_more_than_50k')

plt.subplot(3,1,2)
plt.hist(salary_less_than_50.capitalloss, color = "orange")
plt.title('capitalloss of those with salary_less_than_50k')


capitalloss_describe = pd.DataFrame()

capitalloss_describe['capitalloss of those with salary_more_than_50k'] = salary_more_than_50['capitalloss'].describe()
capitalloss_describe['capitalloss of those with salary_less_than_50k'] = salary_less_than_50['capitalloss'].describe()

capitalloss_describe = capitalloss_describe.drop(['count', '25%', '50%', '75%'])
plt.subplot(3,1,2)
capitalloss_describe.plot.bar()

plt.show() 

Categorical variables wrt salary

In [ ]:
j = 1
categorical_features.remove('Salary') #removing column salary since it is output variable itself. 

for feature in categorical_features:
    df = pd.DataFrame()
    df['More than 50k'] = salary[salary['Salary'] == ' >50K'][feature].value_counts()
    df['Less than 50k'] = salary[salary['Salary'] == ' <=50K'][feature].value_counts()
    
    df.plot.bar(figsize=(14,6))
    
    j = j + 1 

Insight:

Majority of both the groups is employed in private sectors

As educational qualification decreases, no. of people in group of 'salary more than 50k' reduces.

Majority of people having salary more than 50k are married, whereas very small proportion of them is never married, divorced, widow, separated. Significant proportion of people having salary less than 50k are never married, divorced, widow, separated.

Most of the people having salary less than 50k are engaged in manual and low skilled work like craft repair, clerk, transport, machine inspection, farming fishing, handlers cleaners and other services. Whereas people having salary more than 50k are in high skilled works like managers, professors, sales

Insignificant amount of non natives are having salary more than 50k

Data Pre-Processing

In [ ]:
salary1 = salary.copy() 
salary1.drop('Salary',axis=1, inplace =True) 
salary1
     

In [ ]:
#encoding categorical 'input' column
salary1 = pd.get_dummies(salary1)
salary1.head() 

In [ ]:
salary1['Salary'] = salary.Salary
salary1.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
# encoding categorical 'output' column
encoder = LabelEncoder()
encoder.fit(salary1.Salary)

In [ ]:
salary1.Salary = encoder.transform(salary1.Salary)
salary1

Train Test Split

In [ ]:
salary_train.shape #salary.train is original train set provided by ExcelR

In [ ]:
salary_test.shape #salary.test is original train set provided by ExcelR

In [ ]:

#we concatenated both train n test set for EDA and Visualisation.
# now lets separate them. first 30161 entries form train set and rest 15060 entries are test set.

train_set = salary1.iloc[:30161, :]
test_set = salary1.iloc[30161:, :]

In [ ]:
print("Shape of new dataframes - {} , {}".format(train_set.shape, test_set.shape))

In [ ]:
train_set 
     

In [ ]:
x_train = train_set.iloc[:,:102]
x_train 

In [ ]:

y_train = train_set.loc[:,['Salary']]
y_train

In [ ]:
test_set  

In [ ]:
x_test = test_set.iloc[:,:102]
x_test

In [ ]:
y_test = test_set.loc[:,['Salary']]
y_test 

In [ ]:
print("Shape of X_train: ",x_train.shape)
print("Shape of X_test: ", x_test.shape)
print("Shape of y_train: ",y_train.shape)
print("Shape of y_test",y_test.shape)

SVM with default hyper parameters

In [ ]:
classifier = SVC()
classifier.fit(x_train, y_train)

pred = classifier.predict(x_test)

accuracy_score(y_test, pred) 
     

In [ ]:
# GridSearchCV for optimum hyper parameters taking too long to compute.. so we go with default hyper parameters
print(classification_report(y_test, pred)) 

In [ ]:
sns.heatmap(confusion_matrix(y_test, pred), annot=True, fmt='d')

In [ ]:

from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test,pred)
print('AUC-ROC Score:',auc)  